## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
0,0,Labuhan,-6.8844,112.2051,79.61,82,71,6.22,broken clouds,ID,2022-05-16 23:32:30
1,1,Mahebourg,-20.4081,57.7000,75.49,73,75,11.50,broken clouds,MU,2022-05-16 23:32:31
2,2,Wum,6.3833,10.0667,66.54,91,100,2.26,overcast clouds,CM,2022-05-16 23:32:32
3,3,Albany,42.6001,-73.9662,63.27,93,95,10.96,overcast clouds,US,2022-05-16 23:29:33
4,4,Sur,22.5667,59.5289,84.97,37,0,5.57,clear sky,OM,2022-05-16 23:32:33


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
0,0,Labuhan,-6.8844,112.2051,79.61,82,71,6.22,broken clouds,ID,2022-05-16 23:32:30
1,1,Mahebourg,-20.4081,57.7000,75.49,73,75,11.50,broken clouds,MU,2022-05-16 23:32:31
4,4,Sur,22.5667,59.5289,84.97,37,0,5.57,clear sky,OM,2022-05-16 23:32:33
7,7,Manono,-7.3000,27.4167,72.03,67,86,3.04,overcast clouds,CD,2022-05-16 23:32:35
10,10,Atuona,-9.8000,-139.0333,79.11,66,8,15.30,clear sky,PF,2022-05-16 23:32:37


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        262
City           262
Lat            262
Lng            262
Max Temp       262
Humidity       262
Cloudiness     262
Wind Speed     262
Description    262
Country        260
Date           262
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df['Country'] = preferred_cities_df['Country'].fillna(0)
cleaned_df = preferred_cities_df.loc[preferred_cities_df['Country'] != 0]
cleaned_df.count()

C:\Users\trevo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


City_ID        260
City           260
Lat            260
Lng            260
Max Temp       260
Humidity       260
Cloudiness     260
Wind Speed     260
Description    260
Country        260
Date           260
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaned_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Labuhan,ID,79.61,broken clouds,-6.8844,112.2051,
1,Mahebourg,MU,75.49,broken clouds,-20.4081,57.7000,
4,Sur,OM,84.97,clear sky,22.5667,59.5289,
7,Manono,CD,72.03,overcast clouds,-7.3000,27.4167,
10,Atuona,PF,79.11,clear sky,-9.8000,-139.0333,
11,Georgetown,MY,78.73,few clouds,5.4112,100.3354,
12,Bilma,NE,81.16,clear sky,18.6853,12.9164,
20,Black River,JM,80.37,overcast clouds,18.0264,-77.8487,
22,Dargaville,NZ,71.35,broken clouds,-35.9333,173.8833,
29,Faanui,PF,80.19,light rain,-16.4833,-151.7500,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f'{lat},{lng}'

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].fillna(0)
hotel_final_df = hotel_df.loc[hotel_df['Hotel Name'] != 0]
hotel_final_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Labuhan,ID,79.61,broken clouds,-6.8844,112.2051,DoubleFAG
1,Mahebourg,MU,75.49,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
4,Sur,OM,84.97,clear sky,22.5667,59.5289,Sur Plaza Hotel
7,Manono,CD,72.03,overcast clouds,-7.3000,27.4167,Hotel hewa bora
10,Atuona,PF,79.11,clear sky,-9.8000,-139.0333,Villa Enata


In [13]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_final_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>\
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_final_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_final_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[['Lat','Lng']]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))